In [1]:
#uses VectorAdd_#N(_noHR)(_S#noepsE_HRx#hrr)(_NIR)(_IR) - v1.1x

In [2]:
#changed namings
#inReadyC default to simplified definition

In [ ]:
import sys
import math
import os

import import_ipynb
from VectorAdd_noHR_SoE_V1_12 import VectorAdd_noHR_SoE

def MatrixAdd_noHR_SOE(parameters, Print_To_File, Files_Location, Is_Top, modules):
    
    N = parameters["N"]
    M = parameters["M"]
    
    NOEPS= parameters["NOEPS"]
    Enable_Input_Registers = parameters["ENABLE_INPUT_REGISTERS"]
    IN_WIDTH = parameters["IN_WIDTH"]
    #PR<=M
    PR = parameters["PR"]

    if not PR:
        PR=1
    else:
        PR = int(PR)
        if PR>M:
            PR = M
            
    if N==1 or not NOEPS:
        NAdd=N
    else:
        NAdd = int(NOEPS)
        if NAdd<1:
            NAdd=1
        elif NAdd>N:
            NAdd=N

    HRR = math.ceil(N/NAdd);

    if not IN_WIDTH:
        IN_WIDTH=10
    else:
        IN_WIDTH = int(IN_WIDTH)
        if IN_WIDTH<1:
            IN_WIDTH=1

    if not Enable_Input_Registers:
        Enable_Input_Registers=0
    else:
        Enable_Input_Registers = int(Enable_Input_Registers)
        
        
        
    ModuleName="MatrixAdd_{}_{}_{}PV_".format(M, N, PR)
    if(HRR==1):
        ModuleName+="noHR_"
    else:
        ModuleName+="S{}E_HRx{}_".format(NAdd, HRR)     
    if Enable_Input_Registers<=0:
        ModuleName+="N"
    ModuleName+="IR"
    
            
    __Print_To_File = Print_To_File
    if __Print_To_File<=0:
        of=sys.stdout
    else:
        if Is_Top:
            Files_Location += ModuleName+"/"
            if not os.path.exists(Files_Location):
                os.makedirs(Files_Location) 
        of = open(Files_Location+ModuleName+".v", 'w+')

    
    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(", file=of)
    print("parameter IN_WIDTH = {}".format(IN_WIDTH), file=of)
    print(")(", file=of)
    print("input clk, reset, enable,", file=of)
    if HRR>1:
        print("output readyForNewVectorStart,", file=of)
    lghrr=math.ceil(math.log2(HRR))
    if lghrr==1:
        print("output vectorInSeries,", file=of)
    elif lghrr>1:
        print("output [{}:0] vectorInSeries,".format(lghrr-1), file=of)
    RS=math.ceil(M/PR)
    lgRS=math.ceil(math.log2(RS))
    if lgRS<=1:
        print("output reg vectorSetInNo = 0,", file=of)
    else:
        print("output reg [{}:0] vectorSetInNo = 0,".format(lgRS-1), file=of)
    print("input inReady,", file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for j in range(PR):
        for i in range(NAdd):
            print("A{}V{}, ".format(i, j), end='', file=of)
        print(file=of)
    print("input signed [IN_WIDTH-1:0] ", end='', file=of)
    for j in range(PR):
        for i in range(NAdd):
            print("B{}V{}, ".format(i, j), end='', file=of)
        print(file=of)
    ar=RS*PR-M
    ME = PR-ar-1
    if ME==0:
        st1="V0"
    else:
        st1="V0toV{}".format(ME)
    print("output ", end='', file=of)
    if HRR==1:
        print(st1, end='', file=of)
        print("outReady,", file=of)
        NE=NAdd
    else:
        NE = N - NAdd*(HRR-1)
        while NE<=0:
            NE += NAdd
        if NE==1:
            print(st1, end='', file=of)
            print("S0outReady,", file=of)
        else:
            print(st1, end='', file=of)
            print("S0toS{}outReady,".format(NE-1), file=of)
        print("output ", end='', file=of)
        if NE==NAdd:
            print("VSNoutReady1, //not used", file=of)
        elif NE==NAdd-1:
            print(st1, end='', file=of)
            print("S{}outReady,".format(NE), file=of)
        else:
            print(st1, end='', file=of)
            print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    if ar==0:
        print("output reg VNoutReady1 = 0, //not used", file=of)
        print("output reg VNoutReady2 = 0, //not used", file=of)
    else:
        if ME==(PR-2):
            st2="V{}".format(PR-1)
        else:
            st2="V{}toV{}".format(ME+1, PR-1)
        print("output ", end='', file=of)
        if HRR==1:
            print(st2, end='', file=of)
            print("outReady,", file=of)
        else:
            if NE==1:
                print(st2, end='', file=of)
                print("S0outReady,", file=of)
            else:
                print(st2, end='', file=of)
                print("S0toS{}outReady,".format(NE-1), file=of)
            print("output ", end='', file=of)
            if NE==NAdd:
                print("VSNoutReady2, //not used", file=of)
            elif NE==NAdd-1:
                print(st2, end='', file=of)
                print("S{}outReady,".format(NE), file=of)
            else:
                print(st2, end='', file=of)
                print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
    if lghrr==1:
        print("output vectorOutSeries,", file=of)
    elif lghrr>1:
        print("output [{}:0] vectorOutSeries,".format(lghrr-1), file=of)
    if lgRS<=1:
        print("output reg vectorSetOutNo = 1,", file=of)
    else:
        print("output reg [{}:0] vectorSetOutNo = {},".format(lgRS-1, RS-1), file=of)
    print("output signed [IN_WIDTH:0] ", end='', file=of)
    for j in range(PR):
        for i in range(NAdd):
            print("S{}V{}, ".format(i, j), end='', file=of)
        print(file=of)
    print("output ", end='', file=of)
    if HRR==1:
        print(st1, end='', file=of)
        print("earlyOutReady,", file=of)
    else:
        if NE==1:
            print(st1, end='', file=of)
            print("S0earlyOutReady,", file=of)
        else:
            print(st1, end='', file=of)
            print("S0toS{}earlyOutReady,".format(NE-1), file=of)
        print("output ", end='', file=of)
        if NE==NAdd:
            print("CSNearlyOutReady1, //not used", file=of)
        elif NE==NAdd-1:
            print(st1, end='', file=of)
            print("S{}earlyOutReady,".format(NE), file=of)
        else:
            print(st1, end='', file=of)
            print("S{}toS{}earlyOutReady,".format(NE, NAdd-1), file=of)
    if ar==0:
        print("output reg CNearlyOutReady1 = 0, //not used", file=of)
        print("output reg CNearlyOutReady2 = 0 //not used", file=of)
    else:
        print("output ", end='', file=of)
        if HRR==1:
            print(st2, end='', file=of)
            print("earlyOutReady", file=of)
        else:
            if NE==1:
                print(st2, end='', file=of)
                print("S0earlyOutReady,", file=of)
            else:
                print(st2, end='', file=of)
                print("S0toS{}earlyOutReady,".format(NE-1), file=of)
            print("output ", end='', file=of)
            if NE==NAdd:
                print("CSNearlyOutReady2 //not used", file=of)
            elif NE==NAdd-1:
                print(st2, end='', file=of)
                print("S{}earlyOutReady".format(NE), file=of)
            else:
                print(st2, end='', file=of)
                print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
    print(");\n", file=of)
    if RS!=1:
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\tvectorSetInNo <= 0;", file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tif(inReady) begin", file=of)
        if(HRR>1):
            print("\t\t\tif(vectorInSeries=={}) begin".format(HRR-1), file=of)
            print("\t", end='', file=of)
        print("\t\t\tif(vectorSetInNo=={}) begin".format(RS-1), file=of)
        if(HRR>1):
            print("\t", end='', file=of)
        print("\t\t\t\tvectorSetInNo <= 0;", file=of)
        if(HRR>1):
            print("\t", end='', file=of)
        print("\t\t\tend", file=of)
        if(HRR>1):
            print("\t", end='', file=of)
        print("\t\t\telse begin", file=of)
        if(HRR>1):
            print("\t", end='', file=of)
        if RS==2:
            print("\t\t\t\tvectorSetInNo <= 1;", file=of)    
        else:
            print("\t\t\t\tvectorSetInNo <= vectorSetInNo+1;", file=of)
        if(HRR>1):
            print("\t\t\t\tend", file=of)
        print("\t\t\tend", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end", file=of)
        print(file=of)
        
        if ar>0:
            print("wire inReadyC = (vectorSetInNo!={}) & inReady;".format(RS-1), file=of)
            print(file=of)
        
    
    for i in range (PR):
        if(HRR==1):
            if Enable_Input_Registers>0:
                print("VectorAdd_{}_noHR_IR#( .IN_WIDTH(IN_WIDTH) )".format(N), file=of)
            else:
                print("VectorAdd_{}_noHR_NIR#( .IN_WIDTH(IN_WIDTH) )".format(N), file=of)            
        else:
            if Enable_Input_Registers>0:
                print("VectorAdd_{}_S{}E_HRx{}_IR#( .IN_WIDTH(IN_WIDTH) )".format(N, NAdd, HRR), file=of)
            else:
                print("VectorAdd_{}_S{}E_HRx{}_NIR#( .IN_WIDTH(IN_WIDTH) )".format(N, NAdd, HRR), file=of)            
        print("\tVA_{}(clk, reset, enable,".format(i), file=of)
        if(HRR>1):
            if i==0:
                print("\treadyForNewVectorStart,", file=of)
                print("\tvectorInSeries,".format(i), file=of)
            else:
                print("\tLLreadyForNewDataSeries{}, //not used".format(i), file=of)
                print("\tLLinSeries{}, //not used".format(i), file=of)
        if(PR-i<=ar):
            print("\tinReadyC,", file=of)
        else:
            print("\tinReady,", file=of)
        print("\t", end='', file=of)
        for j in range(NAdd):
            print("A{}V{}, ".format(j, i), end='', file=of)
        print(file=of)
        print("\t", end='', file=of)
        for j in range(NAdd):
            print("B{}V{}, ".format(j, i), end='', file=of)
        print(file=of)    
        print("\t", end='', file=of)    
        if i==0:
            if HRR==1:
                print(st1, end='', file=of)
                print("outReady,", file=of)
            else:
                if NE==1:
                    print(st1, end='', file=of)
                    print("S0outReady, ", end='', file=of)
                else:
                    print(st1, end='', file=of)
                    print("S0toS{}outReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("VSNoutReady1, //not used", file=of)
                elif NE==NAdd-1:
                    print(st1, end='', file=of)
                    print("S{}outReady,".format(NE), file=of)
                else:
                    print(st1, end='', file=of)
                    print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
        elif i==PR-ar:
            if HRR==1:
                print(st2, end='', file=of)
                print("outReady,", file=of)
            else:
                if NE==1:
                    print(st2, end='', file=of)
                    print("S0outReady, ", end='', file=of)
                else:
                    print(st2, end='', file=of)
                    print("S0toS{}outReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("CSNoutReady2, //not used", file=of)
                elif NE==NAdd-1:
                    print(st2, end='', file=of)
                    print("S{}outReady,".format(NE), file=of)
                else:
                    print(st2, end='', file=of)
                    print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
        else:
            if HRR==1:
                print("VA{}OR, //not used".format(i), file=of)
            else:
                print("VA{}OR1, VA{}OR2, //both not used".format(i, i), file=of)
        if(HRR>1):
            if i==0:
                print("\tvectorOutSeries,".format(i), file=of)
            else:
                print("\tLLoutSeries{}, //not used".format(i), file=of)
        print("\t", end='', file=of)
        for j in range(NAdd):
            print("S{}V{}, ".format(j, i), end='', file=of)
        print(file=of)
        print("\t", end='', file=of)    
        if i==0:
            if HRR==1:
                print(st1, end='', file=of)
                print("earlyOutReady", file=of)
            else:
                if NE==1:
                    print(st1, end='', file=of)
                    print("S0earlyOutReady, ", end='', file=of)
                else:
                    print(st1, end='', file=of)
                    print("S0toS{}earlyOutReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("VSNearlyOutReady1 //not used", file=of)
                elif NE==NAdd-1:
                    print(st1, end='', file=of)
                    print("S{}earlyOutReady".format(NE), file=of)
                else:
                    print(st1, end='', file=of)
                    print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
        elif i==PR-ar:
            if HRR==1:
                print(st2, end='', file=of)
                print("earlyOutReady", file=of)
            else:
                if NE==1:
                    print(st2, end='', file=of)
                    print("S0earlyOutReady, ", end='', file=of)
                else:
                    print(st2, end='', file=of)
                    print("S0toS{}earlyOutReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("VSNearlyOutReady2 //not used", file=of)
                elif NE==NAdd-1:
                    print(st2, end='', file=of)
                    print("S{}earlyOutReady".format(NE), file=of)
                else:
                    print(st2, end='', file=of)
                    print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
        else:
            if HRR==1:
                print("VA{}EOR //not used".format(i), file=of)
            else:
                print("VA{}EOR1, VA{}EOR2 //both not used".format(i, i), file=of)
        print("\t);\n", file=of)


    for i in range (PR):
        if(HRR==1):
            if Enable_Input_Registers>0:
                print("VectorAdd_{}_noHR_IR#( .IN_WIDTH(IN_WIDTH) )".format(N), file=of)
            else:
                print("VectorAdd_{}_noHR_NIR#( .IN_WIDTH(IN_WIDTH) )".format(N), file=of)            
        else:
            if Enable_Input_Registers>0:
                print("VectorAdd_{}_S{}E_HRx{}_IR#( .IN_WIDTH(IN_WIDTH) )".format(N, NAdd, HRR), file=of)
            else:
                print("VectorAdd_{}_S{}E_HRx{}_NIR#( .IN_WIDTH(IN_WIDTH) )".format(N, NAdd, HRR), file=of)            
        print("\tVA_{}(clk, reset, enable,".format(i), file=of)
        if(HRR>1):
            if i==0:
                print("\treadyForNewVectorStart,", file=of)
                print("\tvectorInSeries,".format(i), file=of)
            else:
                print("\tLLreadyForNewDataSeries{}, //not used".format(i), file=of)
                print("\tLLinSeries{}, //not used".format(i), file=of)
        if(PR-i<=ar):
            print("\tinReadyC,", file=of)
        else:
            print("\tinReady,", file=of)
        print("\t", end='', file=of)
        for j in range(NAdd):
            print("A{}V{}, ".format(j, i), end='', file=of)
        print(file=of)
        print("\t", end='', file=of)
        for j in range(NAdd):
            print("B{}V{}, ".format(j, i), end='', file=of)
        print(file=of)    
        print("\t", end='', file=of)    
        if i==0:
            if HRR==1:
                print(st1, end='', file=of)
                print("outReady,", file=of)
            else:
                if NE==1:
                    print(st1, end='', file=of)
                    print("S0outReady, ", end='', file=of)
                else:
                    print(st1, end='', file=of)
                    print("S0toS{}outReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("VSNoutReady1, //not used", file=of)
                elif NE==NAdd-1:
                    print(st1, end='', file=of)
                    print("S{}outReady,".format(NE), file=of)
                else:
                    print(st1, end='', file=of)
                    print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
        elif i==PR-ar:
            if HRR==1:
                print(st2, end='', file=of)
                print("outReady,", file=of)
            else:
                if NE==1:
                    print(st2, end='', file=of)
                    print("S0outReady, ", end='', file=of)
                else:
                    print(st2, end='', file=of)
                    print("S0toS{}outReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("CSNoutReady2, //not used", file=of)
                elif NE==NAdd-1:
                    print(st2, end='', file=of)
                    print("S{}outReady,".format(NE), file=of)
                else:
                    print(st2, end='', file=of)
                    print("S{}toS{}outReady,".format(NE, NAdd-1), file=of)
        else:
            if HRR==1:
                print("VA{}OR, //not used".format(i), file=of)
            else:
                print("VA{}OR1, VA{}OR2, //both not used".format(i, i), file=of)
        if(HRR>1):
            if i==0:
                print("\tvectorOutSeries,".format(i), file=of)
            else:
                print("\tLLoutSeries{}, //not used".format(i), file=of)
        print("\t", end='', file=of)
        for j in range(NAdd):
            print("S{}V{}, ".format(j, i), end='', file=of)
        print(file=of)
        print("\t", end='', file=of)    
        if i==0:
            if HRR==1:
                print(st1, end='', file=of)
                print("earlyOutReady", file=of)
            else:
                if NE==1:
                    print(st1, end='', file=of)
                    print("S0earlyOutReady, ", end='', file=of)
                else:
                    print(st1, end='', file=of)
                    print("S0toS{}earlyOutReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("VSNearlyOutReady1 //not used", file=of)
                elif NE==NAdd-1:
                    print(st1, end='', file=of)
                    print("S{}earlyOutReady".format(NE), file=of)
                else:
                    print(st1, end='', file=of)
                    print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
        elif i==PR-ar:
            if HRR==1:
                print(st2, end='', file=of)
                print("earlyOutReady", file=of)
            else:
                if NE==1:
                    print(st2, end='', file=of)
                    print("S0earlyOutReady, ", end='', file=of)
                else:
                    print(st2, end='', file=of)
                    print("S0toS{}earlyOutReady, ".format(NE-1), end='', file=of)
                if NE==NAdd:
                    print("VSNearlyOutReady2 //not used", file=of)
                elif NE==NAdd-1:
                    print(st2, end='', file=of)
                    print("S{}earlyOutReady".format(NE), file=of)
                else:
                    print(st2, end='', file=of)
                    print("S{}toS{}earlyOutReady".format(NE, NAdd-1), file=of)
        else:
            if HRR==1:
                print("VA{}EOR //not used".format(i), file=of)
            else:
                print("VA{}EOR1, VA{}EOR2 //both not used".format(i, i), file=of)
        print("\t);\n", file=of)

    if RS!=1:
        print("always @(posedge clk) begin", file=of)
        print("\tif(reset) begin", file=of)
        print("\t\tvectorSetOutNo <= {};".format(RS-1), file=of)
        print("\tend", file=of)
        print("\telse if(enable) begin", file=of)
        print("\t\tif(", end='', file=of)
        if HRR==1:
            print(st1, end='', file=of)
            print("earlyOutReady", end='', file=of)
        else:
            if NE==1:
                print(st1, end='', file=of)
                print("S0earlyOutReady", end='', file=of)
            else:
                print(st1, end='', file=of)
                print("S0toS{}earlyOutReady".format(NE-1), end='', file=of)
        print(") begin", file=of)
        if HRR!=1:
            print("\t\t\tif(vectorOutSeries=={}) begin".format(HRR-1), file=of)
            print("\t", end='', file=of)
        print("\t\t\tif(vectorSetOutNo=={}) begin".format(RS-1), file=of)
        if HRR!=1:
            print("\t", end='', file=of)
        print("\t\t\t\tvectorSetOutNo <= 0;", file=of)
        if HRR!=1:
            print("\t", end='', file=of)
        print("\t\t\tend", file=of)
        if HRR!=1:
            print("\t", end='', file=of)
        print("\t\t\telse begin", file=of)
        if HRR!=1:
            print("\t", end='', file=of)
        if RS==2:
            print("\t\t\t\tvectorSetOutNo <= 1;", file=of)
        else:
            print("\t\t\t\tvectorSetOutNo <= vectorSetOutNo+1;", file=of)
        if HRR!=1:
            print("\t", end='', file=of)
        print("\t\t\tend", file=of)
        if HRR!=1:
            print("\t\t\tend", file=of)
        print("\t\tend", file=of)
        print("\tend", file=of)
        print("end", file=of)
        print(file=of)

    print("endmodule", file=of)

    if __Print_To_File>0:
        of.close()

    #in this cell we will use the inner generator for vecotradd noHR.

    VectorAdd_noHR_SoE(parameters, Print_To_File, Files_Location, 0, modules)
    modules[ModuleName]="MatrixAdd_noHR_SOE"       
    return ModuleName 
